# NTRU Encrypt

O sistema criptográfico NTRU é um sistema de chave pública baseado em reticulados, tendo sido o primeiro deste tipo. 

## Parâmetros

Num sistema criptográfico NTRU, **f** (e **g**, se necessário) são as chaves privadas, enquanto h
é a chave pública. Essas chaves podem ser geradas através do seguinte algoritmo:

## Algoritmo de Geração de chaves

INPUT: Um conjunto de parâmetros $Param = \{N, p, q, d\}$ e uma $seed$.

1. Instanciar $Sampler$ com $\tau(d + 1, d)$ e com a $seed$;
2. $f ← Sampler$
3. se $f$ não é invertível $mod q$ então retornar ao passo 2
4. $g ← Sampler$
5. $h = g/(pf + 1) mod q$

OUTPUT: Chave pública $h$ e chave secreta $(pf, g)$

## Algoritmo de Cifragem

INPUT: Chave pública h, mensagem msg de comprimento mlen, um conjunto de parâmetros $Param$ e a $seed$.
1. $m = Pad(msg, seed)$
2. $rseed = Hash(m|h)$
3. Instanciar $Sampler$ com $T$ e com $rseed$;
4. $r ← Sampler$
5. $t = r ∗ h$
6. $tseed = Hash(t)$
7. Instanciar $Sampler$ com $T$ e com $tseed$;
8. $m_{mask} ← Sampler$
9. $m' = m − m_{mask}(mod p)$
10. $c = t + m$

OUTPUT: Criptograma **c**

O algoritmo acima usa um **método de *padding*** para lidar com
entropia insuficiente potencial de uma mensagem. Supondo que o tamanho da mensagem é válido e menor que $(N - 173)$ *bits*, o algoritmo *padding* funciona da seguinte maneira:

1. Converter $msg$ numa *string* de *bits*. Cada *bit* forma um coeficiente binário para a parte inferior do polinómio $m$, partindo do coeficiente 0.

2. Os últimos $167$ coeficientes de $m (x)$ são escolhidos aleatoriamente de $\{−1, 0, 1\}$ (com um *input* *seed* ). O que dá mais de $256$ bits de entropia.

3. O comprimento da msg é convertido numa *string* binária de 8 *bits* e forma os últimos $173$ a $168$ coeficientes de $m (x)$.

## Algorithm de Decifragem

INPUT: Chave secreta $f$, chave pública $h$, criptograma $c$, e o conjunto de parâmetros $Param$.
1. $m' = f ∗ c (mod p)$
2. $t = c − m$
3. $tseed = Hash(t)$
4. Instanciar $Sampler$ com $T$ e com $tseed$;
5. $m_{mask} = Sampler$
6. $m = m' + m_{mask} (mod p)$
7. $rseed = Hash(m|h)$
8. Instanciar $Sampler$ com $T$ e com $rseed$;
9. $r ← Sampler$
10. $msg, mlen = Extract(m)$
11. se $p · r ∗ h = t$ então
12. $result = msg, mlen$
13. caso contrário
14. $result = ⊥ $

OUTPUT: $result$

No algoritmo acima a operação $Extrair ()$ corresponde ao inverso de $Pad ()$. Emite uma mensagem $m$ e seu comprimento $mlen$.

In [24]:
from hashlib import sha512, pbkdf2_hmac

CRYPTO_BYTES = 32

class NTRU(object):
    def __init__(self, d):
        self.d = d
        self.N = next_prime(1 << self.d)
        self.p = 3
        self.q = next_prime(self.p*self.N)
        Z.<x>  = ZZ[]
        Q.<x>  = PolynomialRing(GF(self.q),name='x').quotient(x^self.N-1)
        self.Q = Q
        self.keygen()
        
    def keygen(self):
        smpl = Sampler(self.N, self.d, self.d+1, None)
        seq = smpl.gen_trinary_string()
        F = self.Q(smpl.gen_trinary_string())       
        f = 1 + self.p*F
        while not f.is_unit():
            F = self.Q(smpl.gen_trinary_string())
            f = 1 + self.p*F
        G = self.Q(smpl.gen_trinary_string())
        g = self.p*G
        h = self.rnd_modq(f^(-1) * g)
        self.chvpriv = (f,g)
        self.chvpub = h
        
    def kem_encap(self, seed):
        encaped_sec = [choice([0,1]) for i in range(8*32)]
        c = self.encrypt(encaped_sec, seed)
        shared_sec = pbkdf2_hmac('sha512', bytes(encaped_sec), bytes(self.chvpub), 100000)
        return c, shared_sec
    
    def kem_decap(self, c):
        encaped_sec = self.decrypt(c)
        shared_sec = pbkdf2_hmac('sha512', bytes(encaped_sec), bytes(self.chvpub), 100000)
        return shared_sec
        
        
    def encrypt(self, msg, seed):
        padded_msg = self.pad_msg(msg, seed)
        m_h = bytes(padded_msg + self.chvpub)
        rseed = int(sha512(m_h).hexdigest(), 16)
        smpl = Sampler(self.N, 0, 0, rseed)
        r = self.Q(smpl.gen_trinary_string())
        t = r*self.Q(self.chvpub)
        tseed = int(sha512(bytes(t)).hexdigest(), 16)
        smpl = Sampler(self.N, 0, 0, tseed)
        msg_mask = self.Q(smpl.gen_trinary_string())
        masked_msg = self.rnd_modp(self.rnd_modq(self.Q(padded_msg) - msg_mask))
        crypt = t + self.Q(masked_msg)
        return crypt

    def decrypt(self, crypt):
        masked_msg = self.rnd_modp(self.rnd_modq(self.chvpriv[0] * crypt))
        t = crypt - self.Q(masked_msg)
        tseed = int(sha512(bytes(t)).hexdigest(), 16)
        smpl = Sampler(self.N, 0, 0, tseed)
        msg_mask = self.Q(smpl.gen_trinary_string())
        padded_msg = self.rnd_modp(self.rnd_modq(self.Q(masked_msg) + msg_mask))
        m_h = bytes(padded_msg + self.chvpub)
        rseed = int(sha512(m_h).hexdigest(), 16)
        smpl = Sampler(self.N, 0, 0, rseed)
        r = self.Q(smpl.gen_trinary_string())
        msg, mlen = self.extract_msg(padded_msg)
        print(msg)
        if self.Q(self.p)*r*self.Q(self.chvpub) == t:
            result = msg[:mlen]
        else:
            result = "Error"
        return result
    
    def rnd_modq(self, l):
        '''Round f mod q
        '''
        qq = (self.q-1)//2
        ll = map(lift,l.list())
        return [n if n <= qq else n - self.q  for n in ll]

    def rnd_modp(self, l):
        '''Round l mod p
        '''
        pp = (self.p-1)//2
        rr = lambda x: x if x <= pp else x - self.p
        return [rr(n%self.p) if n>=0 else -rr((-n)%self.p) for n in l]

    def pad_msg(self, msg, seed):
        ''' Pad message according to NTRU spec
        '''
        mlen = len(msg)
        print(mlen)
        if mlen < (self.N - 173):
            fill_len = self.N-173-mlen+1
            if seed:
                set_random_seed(seed)
            else:
                set_random_seed()
            rand_pad = [choice([-1, 0, 1]) for i in range(167)]
            padded_msg = msg
            padded_msg.extend([0] * fill_len)
            padded_msg.extend(rand_pad)
            for i in range(5):
                padded_msg.append(mlen%2)
                mlen >>= 1
        return padded_msg
            
    def extract_msg(self, padded_msg):
        ''' Reverse padded message
        '''
        msg_len = 0
        for i in range(1,6):
            msg_len = msg_len*2+padded_msg[-i]
        return padded_msg, msg_len
    
    
class Sampler:
    ''' Generate random bit strings of fixed size based on imposed restrictions
    '''
    def __init__(self, N, d, e, seed):
        if seed!=None:
            set_random_seed(seed)
        else:
            set_random_seed() #seed automatically selected
        if d and e:
            self.pool = [-1]*e + [1]*d + [0]*(N-d-e)
            shuffle(self.pool)
        else:
            self.pool = [choice([-1,0,1]) for i in range(0,N)]
    
    def gen_trinary_string(self):
        shuffle(self.pool)
        return self.pool
    
def gen_bstring(n):
    '''Generate a random bit string
    '''
    return [choice([-1,0,1]) for k in range(n)]  

In [25]:
K = NTRU(10)
m = gen_bstring(10)
seed = os.urandom()
e = K.encrypt(m, seed)
d = K.decrypt(e)
print(d)
print(d==e)

TypeError: urandom() takes exactly 1 argument (0 given)

In [26]:
K = NTRU(10)
seed = ZZ.random_element()
c, ss = K.kem_encap(seed)
K.kem_decap(c)

256
[1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

"\xb1\x1ae\xc4\xaf\xeaita+WS\xd5MA\xe4\xd2\xdf\xc3\x90\xa4\xc4TC\xe6\xcbq\x83L'2|\xceZB\x90\xeb\xd4\x0b\x98\x93\xc4cd\x86\xb5\x89\xec\xef\xba\xe4\xbc\xd1=]\xba\x81\x80I/\x93\xc9\xe3k"